### GRU PIPELINE

In [5]:
from sklearn.metrics import f1_score
from pytorch_lightning import Trainer
from datetime import datetime
import torch
import os

import sys
sys.path.insert(0, '/Users/florianrunkel/Documents/02_Uni/04_Masterarbeit/masterthesis/')

from ml_pipe.data.database.mongodb import MongoDb
from ml_pipe.models.gru.dataModule import DataModule
from ml_pipe.models.gru.model import GRUModel

In [8]:
def run_pipeline():
    # Datenquelle initialisieren
    mongo = MongoDb()
    datamodule = DataModule(mongo, batch_size=16)
    datamodule.setup()

    # Modell initialisieren
    input_size = datamodule.train_data[0][0].shape[-1]
    model = GRUModel(input_size=input_size, hidden_size=32)

    # Trainer Setup
    trainer = Trainer(
        max_epochs=10,
        enable_checkpointing=False,
        logger=False,
        enable_model_summary=True,
        log_every_n_steps=10
    )

    # Training
    trainer.fit(model, datamodule=datamodule)

    # Testdaten durchlaufen
    trainer.test(model, datamodule=datamodule)

    # Evaluation: F1 Score manuell berechnen
    all_preds = []
    all_targets = []

    model.eval()
    model.freeze()

    for x, y in datamodule.val_dataloader():
        x = x.to(model.device)
        y = y.to(model.device)

        with torch.no_grad():
            preds = model(x)
            preds = (preds > 0.5).float()

        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(y.cpu().numpy())

    f1 = f1_score(all_targets, all_preds)
    print(f"F1 Score auf Validierungsdaten: {f1:.4f}")

    # Modell speichern
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_path = f"saved_models/gru_model_{timestamp}.pt"

    # Ordner anlegen (falls nicht vorhanden) und Modell speichern
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    torch.save(model.state_dict(), model_path)

    print(f"Modell gespeichert unter: {model_path}")

run_pipeline()

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | gru     | GRU     | 3.5 K  | train
1 | fc      | Linear  | 33     | train
2 | sigmoid | Sigmoid | 0      | train
3 | loss_fn | BCELoss | 0      | train
--------------------------------------------
3.5 K     Trainable params
0         Non-trainable params
3.5 K     Total params
0.014     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 9: 100%|██████████| 37/37 [00:00<00:00, 45.43it/s, train_loss=0.551, train_acc=0.769, val_loss=0.598, val_acc=0.715]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 37/37 [00:00<00:00, 45.38it/s, train_loss=0.551, train_acc=0.769, val_loss=0.598, val_acc=0.715]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 16/16 [00:00<00:00, 139.94it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7272727489471436     │
│         test_loss         │    0.5851530432701111     │
└───────────────────────────┴───────────────────────────┘

F1 Score auf Validierungsdaten: 0.8421
Modell gespeichert unter: saved_models/gru_model_20250408_174259.pt


In [11]:
def predict(input_sequence, model_path="saved_models/gru_model_20250408_174259.pt", input_size=2, hidden_size=32):
    model = GRUModel(input_size=input_size, hidden_size=hidden_size)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    input_tensor = torch.tensor(input_sequence, dtype=torch.float32).unsqueeze(0)  # [1, seq_len, features]
    with torch.no_grad():
        pred = model(input_tensor)
    return float(pred.item()), "wechselbereit" if pred.item() > 0.5 else "bleibt wahrscheinlich"

In [12]:
example_input = [[24, 2], [36, 3], [12, 3]]  # z. B. [Dauer in Monaten, Level]
prob, status = predict(example_input)
print(f"Wechselwahrscheinlichkeit: {prob:.2f} → Einschätzung: {status}")

/var/folders/hw/wbg7dg7934z5jd1dv8g7rxjc0000gn/T/ipykernel_42118/272896914.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


FileNotFoundError: [Errno 2] No such file or directory: 'ml_pipe/models/gru/saved_models/gru_model_20250408_174259.pt'